In [1]:
import sys
from dotenv import load_dotenv
from github import Github
import os

# Create a .env with the following content:
# GH_TOKEN=your_github_token
# REPON=$gh_org/$repo

# Load environment variables from .env file
load_dotenv("../.env")


# Get GitHub token from environment variable
gh_token = os.getenv("GH_TOKEN")
if gh_token is None:
    print("GitHub token not found! Create a .env file a the root with a GH_TOKEN variable.")
    exit(1)
gh_client = Github(gh_token, timeout=60)

repo = os.getenv("REPO") or "gh_org/repo"
repo = gh_client.get_repo(repo)


In [2]:
import pandas as pd
from faker import Faker
import random

# Initialize Faker to generate random data
fake = Faker()

# Set the number of rows for the dataframe
num_rows = 600000

# Generate random IDs and dates
ids = [fake.uuid4() for _ in range(num_rows)]
dates = [fake.date_between(start_date='-1y', end_date='-1m').strftime('%Y-%m-%d') for _ in range(num_rows)]

# Generate random metric values between 0 and 10
metric_values = [round(random.uniform(0, 10),2) for _ in range(num_rows)]
# Generate random country codes
country_codes = [fake.country_code() for _ in range(num_rows)]

# Generate random categories
categories = [random.choice(['Category A', 'Category B', 'Category C']) for _ in range(num_rows)]

# Create the dataframe
ultra_large_df = pd.DataFrame({'unique_key': ids, 'date': dates, 'metric_value': metric_values, 'country_code': country_codes, 'category': categories})

# Print the dataframe
print(ultra_large_df)
local_file_path = 'ultra_large_df.csv'
ultra_large_df.to_csv(local_file_path, index=False)
print('Size of DataFrame in bytes:', os.path.getsize(local_file_path))



                                  unique_key        date  metric_value  \
0       860ed722-b911-4959-870a-a74ab0c5f1dd  2023-01-13          1.80   
1       92ca45d3-e7b2-46ee-b687-b0221d6576c2  2023-03-14          3.87   
2       338dce4a-335d-4299-8eb2-c6d607f0e62c  2022-12-27          5.30   
3       8a4b715c-e66e-421e-b2f8-891ad05a775d  2023-02-24          5.14   
4       f90ce9db-c1ba-4018-a260-1e8c9420138c  2023-03-10          0.12   
...                                      ...         ...           ...   
599995  3ae11d82-997b-4fa2-af16-6a65bd330377  2023-03-13          2.35   
599996  c73ad3a7-4719-42ce-940d-f61bd948b8d4  2023-07-24          9.35   
599997  1d066518-70e3-40d1-9086-2450aee7ab4f  2023-05-14          5.20   
599998  53fa243c-69c5-4904-bfe8-7b8b80aab7b5  2023-04-21          2.20   
599999  4dabb531-c820-4b45-99ec-729cb6e1042f  2022-12-09          6.08   

       country_code    category  
0                AF  Category A  
1                BY  Category A  
2        

In [3]:
import sys
sys.path.append('..')
import importlib
import datagit.github_connector
importlib.reload(datagit.github_connector)
from datagit.github_connector import partition_and_store_metric

## Test with file already existing and splitting new data and historical data
repo = os.getenv("REPO") or "gh_org/repo"
file_path = repo+"/path/to/ultra_large_metric_name19.csv"
partition_and_store_metric(gh_client,  ultra_large_df, file_path)

Partitionning metric...
Storing metric for Month: 2022-09-30 00:00:00
Storing metric...
data-history Samox


/Users/sammyteillet/Documents/Projects/DataDrift/data-drift/tools/datagit/examples/../datagit/dataset_helpers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[column] = formatted_dates


Metric not found, creating it on branch: main
Commit: New data: path/to/ultra_large_metric_name19/2022-09.csv
Metric stored
Storing metric for Month: 2022-10-31 00:00:00
Storing metric...
data-history Samox


/Users/sammyteillet/Documents/Projects/DataDrift/data-drift/tools/datagit/examples/../datagit/dataset_helpers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[column] = formatted_dates


Metric not found, creating it on branch: main
Commit: New data: path/to/ultra_large_metric_name19/2022-10.csv
Metric stored
Storing metric for Month: 2022-11-30 00:00:00
Storing metric...
data-history Samox


...

Metric not found, creating it on branch: main
Commit: New data: path/to/ultra_large_metric_name19/2023-07.csv
Metric stored
Storing metric for Month: 2023-08-31 00:00:00
Storing metric...
data-history Samox


/Users/sammyteillet/Documents/Projects/DataDrift/data-drift/tools/datagit/examples/../datagit/dataset_helpers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[column] = formatted_dates


Metric not found, creating it on branch: main
Commit: New data: path/to/ultra_large_metric_name19/2023-08.csv
Metric stored
Storing metric for Month: 2023-09-30 00:00:00
Storing metric...
data-history Samox


/Users/sammyteillet/Documents/Projects/DataDrift/data-drift/tools/datagit/examples/../datagit/dataset_helpers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[column] = formatted_dates


Metric not found, creating it on branch: main
Commit: New data: path/to/ultra_large_metric_name19/2023-09.csv
Metric stored


In [4]:
import numpy as np

# Set the seed for reproducibility
np.random.seed(42)

ultra_large_df2 = ultra_large_df.copy()

# Select 10 random indices for metric value update
random_indices_metric = np.random.choice(ultra_large_df2.index, size=100, replace=False)
print(random_indices_metric)
# Update metric value with random values between 0 and 10
ultra_large_df2.loc[random_indices_metric, 'metric_value'] = [round(random.uniform(0, 10),2) for _ in range(100)]


[  4242  60608 392832  41643 464234 122681  10258 199077 303125 520722
 589338 285923 340236 339287 170118 520994 591772 550478 455232 123979
 354761 254889 470580 181624 274417 328726 504212 356040  23352 214898
 361248 277103 285786 356048  23550 236107 256942 509629 225513 564335
 184036 354122 408248 516338 367441 413740 222624 558355 350732  39333
 281914 445136 158145 571695  68129 364739 451960 551108 127251 528991
 179870 101684 296154 579076 578190 404752  22682 336015 431689 505472
 470965 267651 454329 439546 337991 326490 521320 586039 460422 513493
  29421 382580  56964  40928  17648 511244  31867 208736 367384 157223
 503369 483668  45568 248656 413335  14685 460849  72643 396418 483375]


In [25]:
ultra_large_df.dtypes

unique_key       object
date             object
metric_value    float64
country_code     object
category         object
dtype: object

In [6]:
import sys
sys.path.append('..')
import importlib

import datagit.drift_evaluators
importlib.reload(datagit.drift_evaluators)
from datagit.drift_evaluators import default_drift_evaluator


import datagit.github_connector
importlib.reload(datagit.github_connector)
from datagit.github_connector import partition_and_store_metric


partition_and_store_metric(gh_client,  ultra_large_df2, file_path)

Partitionning metric...
Storing metric for Month: 2022-09-30 00:00:00
Storing metric...


/Users/sammyteillet/Documents/Projects/DataDrift/data-drift/tools/datagit/examples/../datagit/dataset_helpers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[column] = formatted_dates


Metric found, updating it on branch: main
Content https://raw.githubusercontent.com/Samox/data-history/main/path/to/ultra_large_metric_name19/2022-09.csv?token=ABUWFP7MELFZF44M4Q62HKDFBK3HI
Dataframe dtypes {'unique_key': string[python], 'date': string[python], 'metric_value': string[python], 'country_code': string[python], 'category': string[python]}
Old Dataframe dtypes {'unique_key': string[python], 'date': string[python], 'metric_value': string[python], 'country_code': string[python], 'category': string[python]}
Branch drift/2023-09-20-11-07-03/path-to-ultra-large-metric-name19-202 doesn't exist, creating it...
comparison                                      metric_value      
                                             self other
unique_key                                             
4750f7e9-72d6-40ab-8aa2-6ac15061d591         4.96  6.72
ddae1540-2aae-47ad-8fc7-ed63e4b6f9eb         4.05  0.17
Drift detected
Drift evaluation: {'should_alert': False, 'message': 'Drift detected an

/Users/sammyteillet/Documents/Projects/DataDrift/data-drift/tools/datagit/examples/../datagit/dataset_helpers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[column] = formatted_dates


Metric found, updating it on branch: main
Content https://raw.githubusercontent.com/Samox/data-history/main/path/to/ultra_large_metric_name19/2022-10.csv?token=ABUWFP3NE4KUYYXXPIR6V6TFBK3HS
Dataframe dtypes {'unique_key': string[python], 'date': string[python], 'metric_value': string[python], 'country_code': string[python], 'category': string[python]}
Old Dataframe dtypes {'unique_key': string[python], 'date': string[python], 'metric_value': string[python], 'country_code': string[python], 'category': string[python]}
Branch drift/2023-09-20-11-07-09/path-to-ultra-large-metric-name19-202 doesn't exist, creating it...
comparison                                      metric_value      
                                             self other
unique_key                                             
27bd355e-209e-4cef-aaab-60b40c5f8df3          6.5  1.89
49427e08-99d7-49d6-bb5d-96ab594c11e8         9.25   1.7
82b2c47d-ae15-4565-bcac-590eee69510e         2.18  5.35
e2208052-80f0-4d0c-acae-9e7e6e

...

/Users/sammyteillet/Documents/Projects/DataDrift/data-drift/tools/datagit/examples/../datagit/dataset_helpers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[column] = formatted_dates


Metric found, updating it on branch: main
Content https://raw.githubusercontent.com/Samox/data-history/main/path/to/ultra_large_metric_name19/2023-01.csv?token=ABUWFP5G4ZCMPUNTQWGU5N3FBK3IY
Dataframe dtypes {'unique_key': string[python], 'date': string[python], 'metric_value': string[python], 'country_code': string[python], 'category': string[python]}
Old Dataframe dtypes {'unique_key': string[python], 'date': string[python], 'metric_value': string[python], 'country_code': string[python], 'category': string[python]}
Branch drift/2023-09-20-11-07-27/path-to-ultra-large-metric-name19-202 doesn't exist, creating it...
comparison                                      metric_value      
                                             self other
unique_key                                             
1110da90-77d1-4e94-9f76-5da27d566b6e         9.01  5.89
22d4db18-7bef-4ee4-aa78-4f13777cc32f         2.34  8.78
3967a3b2-681c-48fb-8b1d-fb93b2df2516         0.38  0.26
6b8e6114-125f-45cb-a8ea-112ca5

/Users/sammyteillet/Documents/Projects/DataDrift/data-drift/tools/datagit/examples/../datagit/dataset_helpers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[column] = formatted_dates


Metric found, updating it on branch: main
Content https://raw.githubusercontent.com/Samox/data-history/main/path/to/ultra_large_metric_name19/2023-02.csv?token=ABUWFP5MSWUURD37H522KH3FBK3JE
Dataframe dtypes {'unique_key': string[python], 'date': string[python], 'metric_value': string[python], 'country_code': string[python], 'category': string[python]}
Old Dataframe dtypes {'unique_key': string[python], 'date': string[python], 'metric_value': string[python], 'country_code': string[python], 'category': string[python]}
Branch drift/2023-09-20-11-07-34/path-to-ultra-large-metric-name19-202 doesn't exist, creating it...


GithubException: 422 {"message": "Reference already exists", "documentation_url": "https://docs.github.com/rest/git/refs#create-a-reference"}

In [6]:
# Set the number of rows for the dataframe
num_rows = 6000

# Generate random IDs and dates
ids = [fake.uuid4() for _ in range(num_rows)]
dates = [fake.date_between(start_date='-1m', end_date='today').strftime('%Y-%m-%d') for _ in range(num_rows)]

# Generate random metric values between 0 and 10
metric_values = [round(random.uniform(0, 10),2) for _ in range(num_rows)]
# Generate random country codes
country_codes = [fake.country_code() for _ in range(num_rows)]

# Generate random categories
categories = [random.choice(['Category A', 'Category B', 'Category C']) for _ in range(num_rows)]

# Create the dataframe
new_lines = pd.DataFrame({'unique_key': ids, 'date': dates, 'metric_value': metric_values, 'country_code': country_codes, 'category': categories})

ultra_large_df3 = pd.concat([ultra_large_df2, new_lines], ignore_index=True)

In [24]:
import sys
sys.path.append('..')
import importlib

import datagit.drift_evaluators
importlib.reload(datagit.drift_evaluators)
from datagit.drift_evaluators import default_drift_evaluator


import datagit.github_connector
importlib.reload(datagit.github_connector)
from datagit.github_connector import store_metric

store_metric(gh_client,  ultra_large_df3, file_path, assignees=["Sammy"], store_json=False)

Storing metric...
Metric found, updating it on branch: main
Content https://raw.githubusercontent.com/Samox/data-history/main/path/to/ultra_large_metric_name18.csv?token=ABUWFP5KQIF72QPM6CKZ6VTEW2IZK
Dataframe dtypes {'unique_key': string[python], 'date': string[python], 'metric_value': string[python], 'country_code': string[python], 'category': string[python]}
Old Dataframe dtypes {'unique_key': string[python], 'date': string[python], 'metric_value': string[python], 'country_code': string[python], 'category': string[python]}
New data found
Commit: New data: path/to/ultra_large_metric_name18.csv
https://github.com/Samox/data-history/commit/510cc1a3978ec68cc439c5f19088d9626eb0db2f
Branch metric/path-to-ultra-large-metric-name18-csv doesn't exist, creating it...
comparison                                      metric_value      
                                             self other
unique_key                                             
03aeb072-4c9b-4900-a65f-0cb012459f50         5.28 

In [13]:
ultra_large_df.to_json(local_file_path+'.json')
